In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, count, when, desc

spark = SparkSession.builder.appName("EmployeePerformanceAnalysis").getOrCreate()
df = spark.read.csv("/content/Extended_Employee_Performance_and_Productivity_Data.csv", header=True, inferSchema=True)

1. Read the data into RDD

In [3]:
rdd = df.rdd
rdd.take(5)

[Row(Employee_ID=1, Department='IT', Gender='Male', Age=55, Job_Title='Specialist', Hire_Date=datetime.datetime(2022, 1, 19, 8, 3, 5, 556036), Years_At_Company=2, Education_Level='High School', Performance_Score=5, Monthly_Salary=6750.0, Work_Hours_Per_Week=33, Projects_Handled=32, Overtime_Hours=22, Sick_Days=2, Remote_Work_Frequency=0, Team_Size=14, Training_Hours=66, Promotions=0, Employee_Satisfaction_Score=2.63, Resigned=False),
 Row(Employee_ID=2, Department='Finance', Gender='Male', Age=29, Job_Title='Developer', Hire_Date=datetime.datetime(2024, 4, 18, 8, 3, 5, 556036), Years_At_Company=0, Education_Level='High School', Performance_Score=5, Monthly_Salary=7500.0, Work_Hours_Per_Week=34, Projects_Handled=34, Overtime_Hours=13, Sick_Days=14, Remote_Work_Frequency=100, Team_Size=12, Training_Hours=61, Promotions=2, Employee_Satisfaction_Score=1.72, Resigned=False),
 Row(Employee_ID=3, Department='Finance', Gender='Male', Age=55, Job_Title='Specialist', Hire_Date=datetime.datetime(

2. Department with highest average performance score

In [4]:
df.groupBy("Department") \
  .agg(avg("Performance_Score").alias("avg_performance")) \
  .orderBy(desc("avg_performance")) \
  .show(1)

+-----------+------------------+
| Department|   avg_performance|
+-----------+------------------+
|Engineering|3.0235858289293405|
+-----------+------------------+
only showing top 1 row



3. Department with lowest average employee satisfaction score

In [5]:
df.groupBy("Department") \
  .agg(avg("Employee_Satisfaction_Score").alias("avg_satisfaction")) \
  .orderBy("avg_satisfaction") \
  .show(1)


+----------------+------------------+
|      Department|  avg_satisfaction|
+----------------+------------------+
|Customer Support|2.9605133906990173|
+----------------+------------------+
only showing top 1 row



4. Job title with highest average monthly salary

In [6]:
df.groupBy("Job_Title") \
  .agg(avg("Monthly_Salary").alias("avg_salary")) \
  .orderBy(desc("avg_salary")) \
  .show(1)

+---------+-----------------+
|Job_Title|       avg_salary|
+---------+-----------------+
|  Manager|7801.438741101536|
+---------+-----------------+
only showing top 1 row



5. Probability that an employee has resigned

In [7]:
total_count = df.count()
resigned_count = df.filter(col("Resigned") == "Yes").count()
prob_resigned = resigned_count / total_count
print("P(Resigned) =", prob_resigned)

P(Resigned) = 0.1005467996140238


6. Probability that an employee got a promotion

In [8]:
promoted_count = df.filter(col("Promotions") > 0).count()
prob_promoted = promoted_count / total_count
print("P(Promoted) =", prob_promoted)

P(Promoted) = 0.6674278975018763


7. Average training hours and satisfaction for remote full-time employees

In [9]:
df.filter(col("Remote_Work_Frequency") == "Full-time") \
  .select(avg("Training_Hours").alias("avg_training"),
          avg("Employee_Satisfaction_Score").alias("avg_satisfaction")) \
  .show()

+------------+----------------+
|avg_training|avg_satisfaction|
+------------+----------------+
|        NULL|            NULL|
+------------+----------------+



8. Department with highest promotions per employee

In [10]:
df.groupBy("Department") \
  .agg((avg("Promotions")).alias("avg_promotions")) \
  .orderBy(desc("avg_promotions")) \
  .show(1)

+----------+------------------+
|Department|    avg_promotions|
+----------+------------------+
|     Sales|1.0050149484038962|
+----------+------------------+
only showing top 1 row



 9. Correlation between Monthly Salary and Satisfaction Score

In [11]:
corr = df.stat.corr("Monthly_Salary", "Employee_Satisfaction_Score")
print("Correlation (Salary vs Satisfaction):", corr)

Correlation (Salary vs Satisfaction): 0.0011027716598043247


10. Average sick days for employees with high workload (>45 hrs/week)

In [12]:
df.filter(col("Work_Hours_Per_Week") > 45) \
  .select(avg("Sick_Days").alias("avg_sick_days")) \
  .show()

+------------------+
|     avg_sick_days|
+------------------+
|7.0210225624316545|
+------------------+

